In [7]:
import numpy as np
import pandas as pd
import re

def detect_status(text):
    """Определяет статус по ключевым словам в тексте"""
    if not isinstance(text, str):
        return None
    
    text = text.lower()
    
    # Проверка на "жив" и его формы
    if re.search(r'\bжив(ая|ой)?\b|\bжив[ёуыаоэяию]\b', text):
        return 'жив(а)'
    
    # Проверка на "погиб" и его формы
    elif re.search(r'\bпогиб(ла|ший|шая)?\b|\bпогиб[ёуыаоэяию]\b|\bгибел\b', text):
        return 'погиб(ла)'
    
    # Проверка на "пропал" и его формы
    elif re.search(r'\bпропал(а|и)?\b|\bпропаж(а|и)\b|\bпропавш\b', text):
        return 'пропал(а)'
    
    return None

def process_file(input_file, output_file):
    try:
        # Чтение файла с автоматическим определением кодировки
        try:
            df = pd.read_csv(input_file, encoding='utf-8')
        except UnicodeDecodeError:
            df = pd.read_csv(input_file, encoding='cp1251')
        
        # Проверка необходимых колонок
        required_columns = ['content', 'head', 'status']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            print(f"Ошибка: отсутствуют колонки {missing_columns}")
            return
        
        # Сохраняем оригинальные статусы для статистики
        original_statuses = df['status'].copy()
        
        # Анализируем оба поля и обновляем статус
        for index, row in df.iterrows():
            # Проверяем поле content
            content_status = detect_status(row['content'])
            head_status = detect_status(row['head'])

            if content_status!='пропал(а)' and content_status:
                df.at[index, 'status'] = content_status
            elif head_status:
                df.at[index, 'status'] = head_status
                
        # Статистика изменений
        changed = df['status'] != original_statuses
        print(f"Обработано строк: {len(df)}")
        print(f"Изменено статусов: {changed.sum()}")
        print("\nРаспределение новых статусов:")
       
        
        # Сохранение результата
        df.to_csv(output_file, index=False, encoding='utf-8')
        print(f"\nРезультат сохранён в {output_file}")
        
    except Exception as e:
        print(f"Ошибка обработки: {str(e)}")

# Использование
if __name__ == "__main__":
    input_file = "LizaAlert.csv"    # Укажите ваш входной файл
    output_file = "updated_file.csv"  # Укажите файл для сохранения
    
    process_file(input_file, output_file)
# Проверка наличия столбца status
df=pd.read_csv('updated_file.csv')
if 'status' not in df.columns:
    print("Ошибка: столбец 'status' не найден")
else:
    # Приводим статус к строке и очищаем от пробелов
    df['status'] = df['status'].astype(str).str.strip()
    
    # Условия для замены
    is_empty = (df['status'] == '') | (df['status'].str.lower() == 'nan')
    starts_with_prop = df['status'].str.lower().str.startswith('проп')
    starts_with_stop = df['status'].str.lower().str.startswith('стоп')
    starts_with_dead = df['status'].str.lower().str.startswith(('погиб',))
    starts_with_alive = df['status'].str.lower().str.startswith('жив')
    
    # Объединенные маски
    prop_mask = starts_with_prop | starts_with_stop
    
    # Заменяем значения (теперь без учета gender)
    df.loc[prop_mask, 'status'] = 'пропал(а)'
    df.loc[starts_with_dead, 'status'] = 'погиб(ла)'
    df.loc[starts_with_alive, 'status'] = 'жив(а)'
    
    # Сохранение результата
    df.to_csv('updated_file.csv', index=False)
    print(df['status'].value_counts(dropna=False))

Обработано строк: 40448
Изменено статусов: 35089

Распределение новых статусов:

Результат сохранён в updated_file.csv
status
пропал(а)    24335
жив(а)        9756
nan           3380
погиб(ла)     2977
Name: count, dtype: int64
